In [1]:
############################################################################################################################
### Projeto da Disciplina de PO-233                                                                                        #
### Título: Identificação de dados biométricos utilizando redes Wireless e aprendizado de máquina                          #
### Alunos:                                                                                                                #
###         Ágney Lopes Roth Ferraz                                                                                        #
###	        Alexandre Bellargus Silva da Costa                                                                             #
###	        Carlos Renato de Andrade Figueiredo                                                                            #
###	        Gioliano de Oliveira Braga                                                                                     #
###	        Paulo Ricardo Sousa Fonteles de Castro                                                                         #
###	        Wagner Comin Sonaglio                                                                                          #
############################################################################################################################

## -------------------- 01. Importação de Bibliotecas -------------------- 

Nesta célula são importadas todas as bibliotecas necessárias para o projeto. As bibliotecas estão organizadas por funcionalidade:

Nesta célula, são importadas todas as bibliotecas necessárias para o projeto. As bibliotecas estão organizadas por funcionalidade:

- **Manipulação de dados**: `numpy` e `pandas` para operações numéricas e tabelas (DataFrames).
- **Visualização de dados**: `matplotlib`, `seaborn` e `DecisionBoundaryDisplay` para criação de gráficos, dispersões, fronteiras de decisão e análise visual.
- **Machine Learning**: classificadores (`DecisionTree`, `kNN`, `SVM`, `NaiveBayes`, `RandomForest`, `MLP`, `QDA`), métricas (`accuracy`, `precision`, `recall`, `f1`, `confusion_matrix`, `ROC`, `AUC`) e redução de dimensionalidade com `PCA`.
- **Validação e treinamento**: `train_test_split`, `cross_val_score`, `clone`, usados para dividir os dados e validar os modelos.
- **Visualização de árvores de decisão**: `graphviz` para exportar e renderizar visualmente o modelo de árvore.
- **Utilitários do sistema**: `os`, `pathlib`, `timeit`, `warnings` para controle de diretórios, tempo de execução, caminhos multiplataforma e tratamento de avisos.

Essas importações preparam o ambiente para todas as etapas do aprendizado de máquina: pré-processamento, treino, avaliação e visualização.


In [2]:
# ===============================================================
# Bibliotecas essenciais
# ===============================================================
import os                      # Manipulação de arquivos
from pathlib import Path       # Caminhos multiplataforma
import numpy as np             # Operações numéricas
import pandas as pd            # Manipulação de DataFrames
import warnings                # Controle de avisos

# ===============================================================
# Visualização de dados
# ===============================================================
import seaborn as sns          # Gráficos estatísticos
import matplotlib.pyplot as plt  # Gráficos gerais
from graphviz import Source    # Renderização de árvores .dot
from sklearn.inspection import DecisionBoundaryDisplay  # Fronteiras

# ===============================================================
# Medição de tempo
# ===============================================================
from timeit import default_timer as timer  # Cronômetro

# ===============================================================
# Aprendizado de Máquina
# ---------------------------------------------------------------
# Classificadores
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.base import clone

# Pré-processamento e validação
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV


# Avaliação de desempenho
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report,
    roc_curve, auc
)

# Redução de dimensionalidade
from sklearn.decomposition import PCA

# Geração de dados sintéticos
from sklearn.datasets import make_moons, make_circles, make_classification

# ===============================================================
# Configurações do Projeto
# ===============================================================
from sklearn.exceptions import ConvergenceWarning  # Suprimir avisos do MLP
from config import results_dir, algorithms, cv      # Parâmetros externos


## -------------------- 02. Lista de Databases disponíveis -------------------- 

Lista os arquivos de dados .csv disponíveis para o treino/validação/teste.

In [3]:
# Lista de arquivos .db no diretório atual

db_path = Path('./db')  # Caminho para a pasta db
arquivos_db = list(db_path.glob('*.csv'))  # Procura arquivos .db dentro de /db/

print("Arquivos csv disponíveis na pasta /db/:")
for arquivo in arquivos_db:
    print("-", arquivo.name)  # Usa .name para mostrar só o nome do arquivo

Arquivos csv disponíveis na pasta /db/:
- db_3200.csv
- db_400.csv
- db_4000.csv
- db_800.csv


## -------------------- 03. Escolha e Carregamento do Dataset -------------------- 

Esta etapa realiza o carregamento do conjunto de dados a partir de um arquivo `.csv`. Essa é a primeira etapa prática do pipeline de machine learning, preparando os dados para o pré-processamento e análise.

In [4]:
# Caso os arquivos sejam alterados, deve-se alterar as seguintes linhas
# No item 06: (se necessário - percentual para teste/validação do hold-out)
#               X, y, test_size=0.2, stratify=y, random_state=42
# No item 13:
#               n_experimentos = 10  # total de blocos de experimentos
#               tamanho_exp = 320  # tamanho de cada bloco 

# Diretório base
db_dir = Path('db')

# Caminhos completos para os arquivos
arquivo_treino_validacao = db_dir / 'db_3200.csv'
arquivo_teste_real = db_dir / 'db_800.csv'
arquivo_pca_271_portadoras = db_dir / 'db_400.csv'

# Carregando o banco de treino/validação do hold-out, validação cruzada e leave-one-out
df = pd.read_csv(arquivo_treino_validacao)
df.head()

,gender,rpi1_sc-118,rpi1_sc-111,rpi1_sc2,rpi1_sc32,rpi1_sc67,rpi1_sc106,rpi1_sc120,rpi1_sc121
0,1,521.4720,492.8976,128,38.2884,55.9464,1085.1179,693.6801,669.0209
1,1,397.3109,170.5579,128,143.9514,42.2019,422.0569,216.6010,343.0015
2,1,480.0104,532.8884,128,35.8469,56.3027,1142.7905,626.5693,593.6548
3,1,533.4566,488.0349,128,42.0476,50.3289,1060.5376,688.3204,650.2492
4,1,532.5233,500.6815,128,48.6621,54.6260,1070.5597,698.4512,653.9373


## -------------------- 04. Conversão de Gênero para Valores Numéricos --------------------

Essa função executa o pré-processamento da variável alvo `gender`, convertendo os valores categóricos ('f' e 'm') para valores numéricos (0 e 1), que são aceitos pelos algoritmos de aprendizado de máquina.

In [5]:
# Converte coluna 'gender' de 'f'/'m' para 0/1
def convert_gender_to_numeric(df):
    df['gender'] = df['gender'].replace({'f': 0, 'm': 1})  # troca 'f' por 0 e 'm' por 1
    return df  # retorna o DataFrame com a coluna modificada

## -------------------- 05. Visualização da Dispersão entre Portadoras (Pairplot) -------------------- 

Esta etapa realiza uma análise visual dos dados usando pairplot para verificar a separabilidade entre os gêneros com base nas portadoras. Essa visualização auxilia na etapa de exploração de dados (EDA), ajudando a entender como as variáveis se distribuem e se há padrões visuais úteis para o modelo.

In [6]:
# Cria o diretório 'results' se ainda não existir
Path("results").mkdir(exist_ok=True)

# Define as colunas das portadoras
portadoras = ['rpi1_sc-118', 'rpi1_sc-111', 'rpi1_sc2', 'rpi1_sc32',
              'rpi1_sc67', 'rpi1_sc106', 'rpi1_sc120', 'rpi1_sc121']

# Cria uma cópia do DataFrame com apenas as portadoras e o gênero
df_plot = df[portadoras + ['gender']].copy()

# Converte os valores 0 e 1 para 'Men' e 'Women' (para legenda)
df_plot['gender'] = df_plot['gender'].map({0: 'Women', 1: 'Men'})

# Cria o gráfico de dispersão múltipla (pairplot)
plot = sns.pairplot(df_plot, hue='gender', palette='Set1', corner=True)

# Define o título da figura
plt.suptitle("Carrier Dispersion by Gender", y=1.02)

# Salva o gráfico como imagem
plot.savefig("results/pair_plot.png", dpi=300)
plt.close()

# Mensagem de confirmação
print("Pairplot saved in: results/pair_plot.png")

Pairplot saved in: results/pair_plot.png


## -------------------- 06. Separação do Conjunto de Treino e Validação (80/20) para o Hold-Out --------------------

Nesta etapa, o conjunto de dados completo é dividido em dois subconjuntos: 80% para treino e 20% para validação. Essa técnica é conhecida como **validação hold-out**.

A variável `X` representa as características (atributos) de entrada e `y` é a variável alvo (`gender`). A função `train_test_split` é usada com estratificação para manter a proporção de classes nos dois conjuntos. Esta divisão é usada posteriormente para avaliar o modelo de forma simples, fora dos esquemas de validação cruzada ou LOEO.

> Essa separação só é usada na avaliação final de **validação hold-out**, onde o modelo treinado será testado com 80% dos dados nos 20% restantes.


In [7]:
# Separação do conjunto de treino/validação (80/20)

X = df.drop('gender', axis=1)
y = df['gender']

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print(f"Treino: {X_train.shape}, Validação: {X_val.shape}")

Treino: (2560, 8), Validação: (640, 8)


## -------------------- 07. Treinamento com 80% dos Dados (Hold-Out) --------------------

Neste bloco, os principais classificadores do projeto são treinados utilizando 80% do conjunto original de dados (divisão hold-out). O objetivo é preparar cada modelo individualmente para posterior avaliação com os 20% restantes.
Os modelos utilizados são:
- **kNN** (K-Nearest Neighbors)  
- **Decision Tree** (Árvore de decisão limitada)  
- **Big Tree** (Árvore sem limite de profundidade)  
- **Naive Bayes**  
- **SVM Linear**  
- **SVM RBF** (com kernel radial)


In [8]:
# Define os classificadores a treinar
modelos = {
    "kNN": KNeighborsClassifier(3),
    "Decision Tree": DecisionTreeClassifier(max_depth=5),
    "Big Tree": DecisionTreeClassifier(),
    "Naive Bayes": GaussianNB(),
    "SVM Linear": SVC(kernel="linear", C=0.025, probability=True),
    "SVM RBF": SVC(gamma=2, C=1, probability=True)
}

# Dicionário para armazenar os modelos treinados
modelos_treinados = {}

# Treinamento de cada modelo com os 80% (X_train, y_train)
for nome, modelo in modelos.items():
    clf = clone(modelo)
    clf.fit(X_train, y_train)
    modelos_treinados[nome] = clf
    print(f"Modelo '{nome}' treinado.")

Modelo 'kNN' treinado.
Modelo 'Decision Tree' treinado.
Modelo 'Big Tree' treinado.
Modelo 'Naive Bayes' treinado.
Modelo 'SVM Linear' treinado.
Modelo 'SVM RBF' treinado.


## -------------------- 08. Avaliação Hold-Out (com 20% de validação) --------------------

Nesta etapa, os modelos treinados com 80% dos dados são avaliados com os 20% restantes, utilizando a estratégia Hold-Out.
Para cada classificador (kNN, Decision Tree, Big Tree, Naive Bayes, SVM Linear, SVM RBF), foi gerado um relatório com as métricas de desempenho (precisão, recall, F1-score e suporte).


In [9]:
# Garante que o diretório de resultados exista
Path("results").mkdir(exist_ok=True)

# Caminho do arquivo de saída
holdout_file = Path("results/hold-out.txt")

# Abre o arquivo para escrita
with open(holdout_file, "w") as f:
    f.write("Relatório Hold-Out (20% de validação):\n\n")
    
    for nome, modelo in modelos_treinados.items():
        y_val_pred = modelo.predict(X_val)
        relatorio = classification_report(y_val, y_val_pred, target_names=["Women", "Men"], zero_division=0, digits=4)
        
        # Escreve no arquivo
        f.write(f"Modelo: {nome}\n")
        f.write(relatorio)
        f.write("\n" + "-" * 60 + "\n")
        
        # Exibe no terminal
        print(f"\nModelo: {nome}")
        print(relatorio)
        print("-" * 60)

print("Hold-Out saved in: results/hold-out.txt")


Modelo: kNN
              precision    recall  f1-score   support

       Women     0.9816    1.0000    0.9907       320
         Men     1.0000    0.9812    0.9905       320

    accuracy                         0.9906       640
   macro avg     0.9908    0.9906    0.9906       640
weighted avg     0.9908    0.9906    0.9906       640

------------------------------------------------------------

Modelo: Decision Tree
              precision    recall  f1-score   support

       Women     0.9846    1.0000    0.9922       320
         Men     1.0000    0.9844    0.9921       320

    accuracy                         0.9922       640
   macro avg     0.9923    0.9922    0.9922       640
weighted avg     0.9923    0.9922    0.9922       640

------------------------------------------------------------

Modelo: Big Tree
              precision    recall  f1-score   support

       Women     0.9846    0.9969    0.9907       320
         Men     0.9968    0.9844    0.9906       320

    ac

## -------------------- 09. Avaliação com Validação Cruzada para Todos os Algoritmos -------------------- 

Esta função realiza a avaliação de todos os algoritmos definidos na configuração utilizando validação cruzada estratificada. Essa etapa pertence à fase de avaliação de modelos no pipeline de aprendizado de máquina e permite comparar o desempenho dos classificadores de forma consistente.

In [10]:
# Roda cross_val_score para todos os algoritmos
def generate_cross_val_score_all_alg(df, ycla):
    print("Starting Assessment.")  # mensagem de início

    result = {}  # dicionário para armazenar os resultados

    # percorre todos os algoritmos definidos no dicionário algorithms
    for alg, clf in algorithms.items():
        print(f"Processing {alg}...")  # exibe o nome do algoritmo atual

        try:
            # executa a validação cruzada e armazena os resultados
            result[alg] = cross_val_score(clf, df, ycla, cv=cv)
        except Exception as error:
            # trata e exibe erros que ocorrerem durante a validação
            print(f'Erro in cross validation. \nErro: {error}')

        print("Done.")  # indica que finalizou o algoritmo atual

    print("Assessment Completed")  # mensagem final

    # retorna os resultados como DataFrame
    return pd.DataFrame.from_dict(result)

## -------------------- 10. Geração e Armazenamento dos Resultados com Validação Cruzada -------------------- 

Esta função realiza o pré-processamento do dataset, executa a avaliação dos algoritmos com validação cruzada e salva os resultados estatísticos (média ± desvio padrão) e o gráfico boxplot no diretório especificado. Ela faz parte da fase de avaliação e documentação dos resultados do pipeline de aprendizado de máquina.

In [11]:
# Gera boxplot e salva resultados do cross_val_score
def generate_results(name, df, results_dir=results_dir):
    print(f"Generating Results for the Base {name}.\n")  # início da avaliação

    df = convert_gender_to_numeric(df)  # type: ignore # converte 'gender' para valores numéricos
    ycla = df.gender  # separa variável alvo
    df = df.drop("gender", axis=1)  # remove a coluna 'gender' dos atributos

    start = timer()  # inicia a contagem de tempo
    result = generate_cross_val_score_all_alg(df, ycla)  # roda a avaliação
    time = timer() - start  # calcula tempo de execução

    db_result_dir = 'results'  # define o diretório de saída
    db_result_dir.mkdir(parents=True, exist_ok=True)  # cria o diretório se não existir

    txt_dir = db_result_dir / f"{name}.txt"  # define o caminho do arquivo de texto
    with open(txt_dir, 'w') as file:
        # escreve média ± desvio padrão por algoritmo
        file.write(result.apply(lambda x: "{:.2f} ± {:.2f}".format(x.mean(), x.std())).to_string())
        file.write(f"\nExecution time: {time:.2f} seconds")  # escreve o tempo total

    # cria o gráfico boxplot com os resultados
    plt.boxplot([scores for scores in result.values()])
    plt.xticks(1 + np.arange(result.shape[1]), result.columns)
    plot_file = db_result_dir / f"{name}_boxplot.png"  # define o nome do arquivo da imagem
    plt.savefig(plot_file, dpi=300)  # salva a imagem
    plt.close()  # fecha a figura

    print(f"Saved Information - Fit Time: {time:.2f}.\n")  # fim da execução

## -------------------- 11. Funções Auxiliares de Avaliação e Visualização --------------------

Este bloco define três funções auxiliares utilizadas durante a avaliação dos modelos: o cálculo das métricas (acurácia, precisão, recall e F1-score), a geração de gráficos boxplot dos resultados de validação cruzada e a visualização da matriz de confusão para cada experimento. Essas funções fazem parte da etapa de avaliação quantitativa e visual no pipeline de aprendizado de máquina.

In [12]:
# Calcula métricas de avaliação
def generate_scores(y_test, y_pred):
    return [
        accuracy_score(y_test, y_pred),  # calcula acurácia
        precision_score(y_test, y_pred, zero_division=0),  # calcula precisão
        recall_score(y_test, y_pred, zero_division=0),  # calcula recall
        f1_score(y_test, y_pred, zero_division=0)  # calcula f1-score
    ]

# Plota boxplot com os resultados de acurácia
def plot_boxplot(result: dict, alg_name, boxplot_file, y_label_text="Score"):
    df_result = pd.DataFrame.from_dict(result)  # transforma o dicionário em DataFrame
    
    # Garante que o rótulo do eixo Y seja uma string
    if not isinstance(y_label_text, str):
        print(f"Warning: y_label_text era {type(y_label_text)}. Convertendo.")
        y_label_text = str(y_label_text)

    plt.style.use('default')  # aplica o estilo padrão do matplotlib
    plt.figure(figsize=(8, 5))  # define o tamanho da figura
    plt.boxplot([scores for scores in df_result.values.T])  # plota os boxplots
    plt.title("Leave-One-Experiment-Out Boxplot")  # título do gráfico
    plt.ylabel(y_label_text)  # rótulo do eixo Y
    plt.xticks(1 + np.arange(df_result.shape[1]), df_result.columns)  # define os rótulos no eixo X
    plt.grid(True, linestyle='--', alpha=0.7)  # adiciona grid para melhor legibilidade
    plt.savefig(boxplot_file, dpi=300)  # salva a imagem no caminho especificado
    plt.close()  # fecha o gráfico
    print(f"Boxplot salvo em: {boxplot_file}")  # mensagem de confirmação

# Plota a matriz de confusão para um experimento
def plot_heatmap(Y_test, Y_pred, alg_name, db_result_dir, index):
    cm = confusion_matrix(Y_test, Y_pred, labels=[0, 1])  # gera matriz de confusão 2x2

    plt.figure(figsize=(10, 8))  # define o tamanho da figura
    sns.heatmap(cm, xticklabels=["Women", "Men"], yticklabels=["Women", "Men"],
                cmap="YlGnBu", annot=True, fmt="d")  # plota a matriz com cores e valores
    sns.set(font_scale=1.1)  # ajusta o tamanho da fonte
    plt.xlabel("Predicted Classes")  # rótulo eixo X
    plt.ylabel("True Classes")  # rótulo eixo Y
    plt.title(f"Confusion Matrix - {alg_name}")  # título do gráfico
    plot_file = db_result_dir / f"{alg_name}_confusion_matrix_experiment{index+1}.png"  # caminho de saída
    plt.savefig(plot_file, dpi=300)  # salva a imagem
    plt.close()  # fecha o gráfico

## -------------------- 12. Função para Formatação de Resultados Estatísticos --------------------

Esta função auxilia na apresentação dos resultados, formatando a média e o desvio padrão de uma métrica de avaliação (como acurácia, precisão, etc.) em uma string padronizada. É utilizada na geração dos arquivos de saída para facilitar a leitura dos resultados.

In [13]:
# Formata string de média ± desvio padrão
def format_string(valor, scores):
    return f"Mean {valor}: {np.mean(scores):.4f} ± {np.std(scores):.4f}\n"  # retorna string formatada com média e desvio padrão

## -------------------- 13. Avaliação Leave-One-Experiment-Out (LOEO) para um Modelo -------------------- 

Esta função realiza a avaliação de um modelo usando a técnica Leave-One-Experiment-Out (LOEO), onde o conjunto de dados é dividido em 10 blocos (experimentos). Em cada iteração, um bloco é usado como teste e os demais como treino. Para cada rodada, o modelo é treinado, testado e suas métricas armazenadas. Ao final, gera gráficos, salva as métricas em arquivo e retorna os resultados consolidados. Esta etapa pertence à fase de avaliação robusta no pipeline de aprendizado de máquina.

In [14]:
# Avaliação leave-one-experiment-out para um modelo
def leave_one_experiment_out_evaluation(df, best_alg_name, results_dir, name):
    print(f"\nStarting leave-one-experiment-out evaluation with the best model: {best_alg_name}")  # início da avaliação

    n_experimentos = 10  # total de blocos de experimentos
    scores = {"Accuracy": [], "Precision": [], "Recall": [], "F1-Score": []}  # dicionário para guardar métricas
    y_true_all, y_pred_all = [], []  # listas para acumular rótulos reais e preditos

    db_result_dir = results_dir / name  # define pasta de saída
    db_result_dir.mkdir(parents=True, exist_ok=True)  # cria pasta se não existir
    boxplot_file = db_result_dir / f"{name}_leave_one_exp_boxplot.png"  # caminho do gráfico final

    # Configuração do StratifiedKFold para garantir balanceamento
    skf = StratifiedKFold(n_splits=n_experimentos)
    
    # Convertendo dados uma única vez
    df = convert_gender_to_numeric(df.copy())
    X = df.drop("gender", axis=1)
    y = df["gender"]
    
    # Loop sobre os folds estratificados
    for i, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        print(f"Processing experiment {i+1}/{n_experimentos}...")
        
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # Normalização dos dados
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        model = clone(algorithms[best_alg_name])  # clona o melhor modelo
        model.fit(X_train, y_train)  # treina o modelo
        y_pred = model.predict(X_test)  # realiza a predição

        acc, precision, recall, f1 = generate_scores(y_test, y_pred)  # calcula métricas
        scores["Accuracy"].append(acc)  # armazena acurácia
        scores["Precision"].append(precision)  # armazena precisão
        scores["Recall"].append(recall)  # armazena recall
        scores["F1-Score"].append(f1)  # armazena f1-score

        y_true_all.extend(y_test)  # acumula rótulos reais
        y_pred_all.extend(y_pred)  # acumula predições

        print(f"Experiment {i+1}: Accuracy = {acc:.4f}")  # exibe acurácia do experimento
        plot_heatmap(y_test, y_pred, best_alg_name, db_result_dir, i)  # plota matriz de confusão

    # Plota boxplot com as acurácias dos 10 experimentos
    plot_boxplot({"Accuracy": scores["Accuracy"]}, best_alg_name, boxplot_file, "Accuracy")

    # Salva os resultados em arquivo de texto
    txt_file = db_result_dir / f"{name}_leave_one_exp_scores.txt"
    with open(txt_file, 'w') as f:
        f.write("Accuracies leave-one-experiment-out:\n")
        for i, score in enumerate(scores["Accuracy"]):
            f.write(f"Experiment {i+1}: {score:.4f}\n")
        for key, score in scores.items():
            f.write(format_string(key, score))

    print(f"Scores saved in: {txt_file}\n")  # confirmação
    return scores  # retorna o dicionário de métricas

## -------------------- 15. Avaliação Leave-One-Experiment-Out para Todos os Modelos --------------------

Esta função executa a avaliação LOEO para todos os algoritmos definidos na configuração. Cada modelo é avaliado separadamente, e as acurácias de cada experimento são armazenadas. Ao final, um gráfico boxplot é gerado comparando os desempenhos entre os modelos. Esta etapa pertence à fase de comparação e análise de desempenho entre os classificadores no pipeline de aprendizado de máquina.

In [15]:
# Executa avaliação leave-one-out para todos os modelos disponíveis
def leave_one_experiment_out_evaluation_all_models(db_name):
    print("\nStarting leave-one-experiment-out evaluation for all models...")
    
    acc_results = {}  # dicionário para guardar resultados de acurácia por modelo
    results_df = pd.DataFrame()  # DataFrame para armazenar todos os resultados
    
    # Carrega os dados uma única vez
    df = pd.read_csv(db_name)
    
    # Loop por todos os algoritmos
    for i, alg_name in enumerate(algorithms.keys(), 1):
        print(f"\nEvaluating model {alg_name} ({i}/{len(algorithms)})...")
        
        # Gera nome para pasta/arquivos
        name = Path(db_name).stem.replace(".", "_") + f"_{alg_name}"
        
        # Executa avaliação LOEO
        scores = leave_one_experiment_out_evaluation(df, alg_name, results_dir, name)
        
        # Armazena resultados
        acc_results[alg_name] = scores["Accuracy"]
        results_df[alg_name] = scores["Accuracy"]
        
        # Salva resultados intermediários
        results_df.to_csv(results_dir / "loo_all_models_results.csv")
        print(f"Intermediate results saved for {alg_name}")

    # Gera gráfico comparativo final
    boxplot_file = results_dir / "acc_boxplot_all_models.png"
    plot_boxplot(acc_results, "All Models", boxplot_file, "Accuracy")
    
    # Exibe resumo estatístico
    print("\nMean Accuracy by Model:")
    print(results_df.mean().sort_values(ascending=False))
    
    print("\nLeave-one-experiment-out evaluation completed for all models.")

## -------------------- 16. Execução Final da Avaliação para Todos os Modelos --------------------

Este bloco define o caminho do dataset `.csv` e executa a função de avaliação Leave-One-Experiment-Out para todos os modelos disponíveis. Ao final, imprime os arquivos gerados na pasta `results/`, encerrando a fase principal de avaliação do pipeline de aprendizado de máquina.

In [ ]:
# Executa a avaliação LOEO para todos os modelos
leave_one_experiment_out_evaluation_all_models(arquivo_treino_validacao)

# Lista os arquivos gerados na pasta de resultados
print("\nResults generated in:")
print([f for f in os.listdir('./results') if f.endswith(('.png', '.txt', '.csv'))])



Starting leave-one-experiment-out evaluation for all models...

Evaluating model kNN (1/6)...

Starting leave-one-experiment-out evaluation with the best model: kNN
Processing experiment 1/10...
Experiment 1: Accuracy = 0.9969
Processing experiment 2/10...
Experiment 2: Accuracy = 0.9938
Processing experiment 3/10...
Experiment 3: Accuracy = 0.9938
Processing experiment 4/10...
Experiment 4: Accuracy = 1.0000
Processing experiment 5/10...
Experiment 5: Accuracy = 0.9844
Processing experiment 6/10...
Experiment 6: Accuracy = 0.9938
Processing experiment 7/10...


## -------------------- 17. Avaliação Final com Todos os Dados Usando o Melhor Modelo --------------------

Esta etapa realiza a avaliação final do melhor modelo identificado pela média de acurácia na validação cruzada. O modelo é treinado com todos os dados disponíveis e, em seguida, avaliado no mesmo conjunto para fins descritivos. São geradas as métricas detalhadas (precisão, recall, f1-score e suporte) e a matriz de confusão final, que mostram como o modelo se comporta globalmente. A matriz é salva em `results/melhor_modelo_confusion_matrix_FINAL.png`.

In [ ]:
# ===================== CARREGAMENTO DOS DADOS =====================
df = pd.read_csv(arquivo_treino_validacao)
X = df.drop('gender', axis=1)
y = df['gender']

# ===================== CONFIGURAÇÃO DE VALIDAÇÃO =====================
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

param_grid = {
    'max_depth': [3, 5, 7, None],
    'criterion': ['gini', 'entropy']
}

# ===================== GRID SEARCH ANINHADO =====================
grid_search = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid,
    cv=inner_cv,
    scoring='accuracy'
)

nested_scores = cross_val_score(grid_search, X=X, y=y, cv=outer_cv)

print("=== Validação Cruzada Aninhada ===")
print(f"Acurácia Média: {nested_scores.mean():.4f} ± {nested_scores.std():.4f}")

# ===================== TREINAMENTO FINAL =====================
grid_search.fit(X, y)
melhor_modelo = grid_search.best_estimator_
print(f"\nMelhores parâmetros: {grid_search.best_params_}")

# ===================== RELATÓRIO FINAL =====================
y_pred = melhor_modelo.predict(X)
relatorio = classification_report(y, y_pred, target_names=["Women", "Men"], digits=4)
print("\nRelatório Final do Melhor Modelo (em todos os dados):")
print(relatorio)

# ===================== SALVA ARQUIVOS =====================
Path("results").mkdir(exist_ok=True)
joblib.dump(melhor_modelo, 'results/melhor_modelo.pkl')

with open("results/validacao_cruzada_aninhada.txt", "w", encoding="utf-8") as f:
    f.write("=== Validação Cruzada Aninhada ===\n")
    f.write(f"Acurácia Média: {nested_scores.mean():.4f} ± {nested_scores.std():.4f}\n\n")
    f.write(f"Melhores Parâmetros: {grid_search.best_params_}\n\n")
    f.write(relatorio)
print("Relatório salvo em: results/validacao_cruzada_aninhada.txt")

# ===================== MATRIZ DE CONFUSÃO =====================
cm = confusion_matrix(y, y_pred, labels=[0, 1])
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Women", "Men"],
            yticklabels=["Women", "Men"])
plt.xlabel("Classe Predita")
plt.ylabel("Classe Real")
titulo = f"Matriz de Confusão - Melhor Modelo Final ({melhor_modelo.__class__.__name__})"
plt.title(titulo)
plt.tight_layout()
plt.savefig("results/melhor_modelo_confusion_matrix_final.png", dpi=300)
plt.close()
print("Matriz de confusão salva em: results/melhor_modelo_confusion_matrix_final.png")

=== Validação Cruzada Aninhada ===
Acurácia Média: 0.9916 ± 0.0027

Melhores parâmetros: {'criterion': 'gini', 'max_depth': 5}

Relatório Final do Melhor Modelo (em todos os dados):
              precision    recall  f1-score   support

       Women     0.9981    1.0000    0.9991      1600
         Men     1.0000    0.9981    0.9991      1600

    accuracy                         0.9991      3200
   macro avg     0.9991    0.9991    0.9991      3200
weighted avg     0.9991    0.9991    0.9991      3200

Relatório salvo em: results/validacao_cruzada_aninhada.txt
Matriz de confusão salva em: results/melhor_modelo_confusion_matrix_final.png


## -------------------- 18. Comparação Visual entre Classificadores --------------------

Adicionalmente, é gerado um gráfico com a comparação visual dos limites de decisão de vários algoritmos de classificação em datasets sintéticos. Esse gráfico ilustra a capacidade de generalização e a forma como cada modelo segmenta o espaço de decisão. Faz parte da etapa de análise comparativa/visualização.

In [ ]:
# Classificadores utilizados no projeto
names = [
    "kNN", "Decision Tree", "Big Tree", "Naive Bayes", "SVM Linear", "SVM RBF"
]

classifiers = [
    KNeighborsClassifier(3),  # kNN
    DecisionTreeClassifier(max_depth=5),  # Árvore pequena
    DecisionTreeClassifier(),  # Árvore grande
    GaussianNB(),  # Naive Bayes
    SVC(kernel="linear", C=0.025, probability=True),  # SVM Linear
    SVC(gamma=2, C=1, probability=True),  # SVM RBF
]

# Datasets sintéticos para visualização
datasets = [
    make_moons(noise=0.3, random_state=0),
    make_circles(noise=0.2, factor=0.5, random_state=1),
    make_classification(
        n_features=2, n_redundant=0, n_informative=2,
        random_state=42, n_clusters_per_class=1
    )
]

# Ignora avisos de convergência (MLP, etc)
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)

    # Cria figura com várias subplots
    figure = plt.figure(figsize=(20, 6))
    i = 1

    for ds in datasets:
        X, y = ds
        X = StandardScaler().fit_transform(X)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=42)

        # Primeira coluna: dados originais
        ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
        ax.set_title("Input data")
        ax.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.coolwarm, edgecolors='k')
        i += 1

        # Colunas seguintes: classificadores
        for name, clf in zip(names, classifiers):
            ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
            clf.fit(X_train, y_train)
            score = clf.score(X_test, y_test)

            # Exibe fronteira de decisão
            display = DecisionBoundaryDisplay.from_estimator(
                clf, X, response_method="predict", cmap=plt.cm.coolwarm, alpha=0.8, ax=ax
            )
            ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=plt.cm.coolwarm, edgecolors='k')
            ax.set_title(f"{name}\nAccuracy: {score:.2f}")
            i += 1

    plt.tight_layout()
    plt.savefig("results/classifier_comparison.png", dpi=300)
    plt.close()
    print("Comparison Graph saved in: results/classifier_comparison.png")

Comparison Graph saved in: results/classifier_comparison.png


## -------------------- 19. Geração da Árvore de Decisão Geral com Todos os Dados --------------------

Este bloco treina uma árvore de decisão com todos os dados disponíveis e gera uma representação visual completa da estrutura da árvore. Esta etapa complementa a avaliação, permitindo uma interpretação visual do modelo final e das variáveis mais relevantes, sendo parte da explicação e apresentação de resultados do projeto.

In [ ]:
# Garante que os dados estejam prontos
df_full = convert_gender_to_numeric(df.copy())  # converte gênero para 0 e 1
X = df_full.drop("gender", axis=1)  # separa atributos
y = df_full["gender"]  # variável alvo

# Treina o modelo completo com todos os dados
modelo_geral = DecisionTreeClassifier(
    max_depth=5,  # profundidade máxima da árvore
    criterion='entropy',  # critério de divisão
    random_state=42  # semente para reprodutibilidade
)
modelo_geral.fit(X, y)  # treina a árvore com todos os dados

# Garante que a pasta 'results' exista
Path("results").mkdir(exist_ok=True)  # cria a pasta se não existir

# Exporta a árvore para um arquivo .dot
export_graphviz(
    modelo_geral,
    out_file="results/credit_tree_geral.dot",  # caminho do arquivo de saída
    feature_names=X.columns,  # nomes das variáveis
    class_names=["Women", "Men"],  # nomes das classes
    rounded=True,  # cantos arredondados
    filled=True  # cores nas folhas
)

# Renderiza o arquivo .dot e salva como .png
graph = Source.from_file("results/credit_tree_geral.dot")  # carrega o arquivo .dot
graph.render("results/credit_tree_geral", format="png", cleanup=True)  # salva como imagem

# Mensagem final de confirmação
print("General Tree saved in: results/credit_tree_geral.png")

General Tree saved in: results/credit_tree_geral.png


## -------------------- 20. Curva ROC e Visualização PCA dos Dados --------------------

Este bloco complementa a análise dos resultados com dois gráficos. Primeiro, a Curva ROC (Receiver Operating Characteristic), que mostra a performance do modelo ao variar o limiar de decisão, junto com o valor da AUC (Área sob a curva). Em seguida, um gráfico de dispersão usando PCA (Análise de Componentes Principais), reduzindo os dados para 2 dimensões e colorindo por gênero, permitindo visualizar possíveis separações entre as classes.

In [ ]:
# Função para plotar a curva ROC com AUC
def plot_roc_curve(model, X_test, y_test, label="Modelo"):
    # Verifica se o modelo possui método predict_proba ou decision_function
    if hasattr(model, "predict_proba"):
        probs = model.predict_proba(X_test)[:, 1]  # usa probabilidade da classe positiva
    else:
        probs = model.decision_function(X_test)  # usa função de decisão (ex: SVM)

    # Calcula os valores de FPR (falsos positivos) e TPR (verdadeiros positivos)
    fpr, tpr, _ = roc_curve(y_test, probs)

    # Calcula a área sob a curva (AUC)
    roc_auc = auc(fpr, tpr)

    # Cria o gráfico
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'{label} (AUC = {roc_auc:.2f})')  # curva principal
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')  # linha de referência (aleatório)
    plt.xlabel('FPR (False Positives)')  # eixo X
    plt.ylabel('TPR (True Positives)')  # eixo Y
    plt.title('ROC Curve')  # título
    plt.legend(loc="lower right")  # legenda
    plt.grid()  # adiciona grade
    plt.savefig("results/roc_curve.png", dpi=300)  # salva o gráfico
    print("ROC Curve saved in: results/roc_curve.png") 
    plt.close()  # fecha figura

# Plota a curva ROC para o modelo geral
plot_roc_curve(modelo_geral, X, y, label="General Tree")

# Reduz os dados para 2D com PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)  # aplica a transformação PCA

# Converte rótulos numéricos para texto
labels = y.map({0: 'Women', 1: 'Men'})

# Cria gráfico de dispersão por gênero
plt.figure(figsize=(8, 6))
for label in labels.unique():
    idx = labels == label
    plt.scatter(X_pca[idx, 0], X_pca[idx, 1], label=label, alpha=0.7)  # plota cada grupo

plt.title("PCA Dispersion by Gender - 8 subcarriers")  # título do gráfico
plt.xlabel("Principal Component 1")  # eixo X
plt.ylabel("Principal Component 2")  # eixo Y
plt.legend()  # exibe legenda
plt.savefig("results/pca_2d_scatter.png", dpi=300)  # salva o gráfico
print("PCA 2D Scatter saved in: results/pca_2d_scatter.png") 
plt.close()  # fecha a figura

ROC Curve saved in: results/roc_curve.png
PCA 2D Scatter saved in: results/pca_2d_scatter.png


## -------------------- 21. PCA com Visualização Bidimensional (8 e 271 subportadoras) --------------------

Essa etapa realiza uma redução de dimensionalidade via PCA para facilitar a visualização dos dados em 2D. São usados dois conjuntos diferentes (um com 271 subportadoras e outro com 8), permitindo comparação visual entre os gêneros após a transformação.

In [ ]:
# === PCA - Visualização com 271 subportadoras ===

df = pd.read_csv(arquivo_pca_271_portadoras)  # carrega o CSV como DataFrame

y = df.iloc[:, 0]  # separa a variável alvo (0 = mulher, 1 = homem)
X = df.iloc[:, 1:]  # separa os atributos (colunas de subportadoras)

scaler = StandardScaler()  # instancia normalizador (zero média, variância 1)
X_scaled = scaler.fit_transform(X)  # aplica normalização aos dados

labels = y.map({0: 'Men', 1: 'Women'})  # converte rótulos numéricos em texto

pca = PCA(n_components=2)  # define PCA com 2 componentes principais
X_pca = pca.fit_transform(X_scaled)  # reduz os dados para 2D

plt.figure(figsize=(8, 6))  # define tamanho da figura
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1],
                hue=y.map({0: 'Women', 1: 'Men'}), palette='Set1')  # cria gráfico colorido por gênero
plt.title('PCA Dispersion by Gender - 271 subcarriers')  # título do gráfico
plt.xlabel("Principal Component 1")  # legenda eixo X
plt.ylabel("Principal Component 2")  # legenda eixo Y
plt.legend()  # exibe legenda
plt.grid(True)  # exibe grade no gráfico
plt.tight_layout()  # ajusta margens
plt.savefig("results/pca_271p.png", dpi=300)  # salva a imagem no diretório
print("PCA 271p: results/pca_271p.png")
plt.close()  # fecha a figura para liberar memória


# === PCA - Visualização com 8 subportadoras ===

df = pd.read_csv(arquivo_treino_validacao)  # carrega o CSV como DataFrame

y = df.iloc[:, 0]  # variável alvo (0 = mulher, 1 = homem)
X = df.iloc[:, 1:]  # atributos

scaler = StandardScaler()  # normalizador padrão
X_scaled = scaler.fit_transform(X)  # aplica normalização

labels = y.map({0: 'Men', 1: 'Women'})  # transforma 0/1 em texto

pca = PCA(n_components=2)  # PCA para 2 dimensões
X_pca = pca.fit_transform(X_scaled)  # aplica a transformação PCA

plt.figure(figsize=(8, 6))  # tamanho da figura
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1],
                hue=y.map({0: 'Women', 1: 'Men'}), palette='Set1')  # gráfico com cores por gênero
plt.title('PCA Dispersion by Gender - 8 subcarriers')  # título do gráfico
plt.xlabel("Principal Component 1")  # eixo X
plt.ylabel("Principal Component 2")  # eixo Y
plt.legend()  # legenda
plt.grid(True)  # exibe grade
plt.tight_layout()  # ajusta layout
plt.savefig("results/pca_8p.png", dpi=300)  # salva imagem no diretório results
print("PCA 8p: results/pca_8p.png")
plt.close()  # fecha a figura

PCA 271p: results/pca_271p.png
PCA 8p: results/pca_8p.png


## -------------------- 22. Relatório Final com Banco de Teste Real --------------------

Esta etapa realiza a avaliação final do melhor modelo usando o conjunto de **teste real**.
o modelo é treinado com **todo o conjunto de treino/validação** e testado contra os dados reais finais.

In [ ]:
# Carrega dados reais de teste
df_teste_real = pd.read_csv(arquivo_teste_real)
X_real = df_teste_real.drop('gender', axis=1)
y_real = df_teste_real['gender']

# Treina novamente o melhor modelo com 100% dos dados de treino/validação
X_full = df.drop('gender', axis=1)
y_full = df['gender']
melhor_modelo.fit(X_full, y_full)

# Predição no conjunto real
y_pred_real = melhor_modelo.predict(X_real)

# Geração do relatório
relatorio_real = classification_report(y_real, y_pred_real, target_names=["Women", "Men"], digits=4)

# Exibe no terminal
print("\nRelatório de Classificação com Dados Reais:")
print(relatorio_real)

# Salva em arquivo .txt
Path("results").mkdir(exist_ok=True)
with open("results/relatorio_teste_real.txt", "w", encoding="utf-8") as f:
    f.write("Relatório de Classificação com Dados Reais:\n\n")
    f.write(relatorio_real)
print("Relatório salvo em: results/relatorio_teste_real.txt")

# Matriz de confusão
cm_real = confusion_matrix(y_real, y_pred_real, labels=[0, 1])
plt.figure(figsize=(8, 6))
sns.heatmap(cm_real,
            annot=True,
            fmt="d",
            cmap="Blues",
            xticklabels=["Women", "Men"],
            yticklabels=["Women", "Men"])
plt.xlabel("Classe Predita")
plt.ylabel("Classe Real")
plt.title(f"Matriz de Confusão - Teste Real ({melhor_modelo.__class__.__name__})")

# Salva imagem
plt.tight_layout()
plt.savefig("results/melhor_modelo_confusion_matrix_teste_real.png", dpi=300)
plt.close()
print("Real Test Confusion Matrix saved in: results/melhor_modelo_confusion_matrix_teste_real.png")


Relatório de Classificação com Dados Reais:
              precision    recall  f1-score   support

       Women     0.9797    0.9650    0.9723       400
         Men     0.9655    0.9800    0.9727       400

    accuracy                         0.9725       800
   macro avg     0.9726    0.9725    0.9725       800
weighted avg     0.9726    0.9725    0.9725       800

Relatório salvo em: results/relatorio_teste_real.txt
Real Test Confusion Matrix saved in: results/melhor_modelo_confusion_matrix_teste_real.png


## -------------------- Fim do código --------------------